In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "movie": "Jurassic Park",
        "answer": "Emojis 🦖⚠️🏝️ represent dinosaurs, danger, and a remote island.",
    },
    {
        "movie": "The Matrix",
        "answer": "Emojis 💻🔫🕶️ represent computers, action, and the iconic sunglasses of the characters.",
    },
    {
        "movie": "Inception",
        "answer": "Emojis 🌀💤🧠 represent dreams, sleep, and mind-bending concepts.",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert that knows all informations about movies. With given movie title, you answer three emojis that represent the movie."),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)


In [2]:
invoke_chain("The Grand Budapest Hotel")


content='Emojis 🏨🎩🔎 represent a hotel, elegance, and mystery.'


In [3]:
invoke_chain("Blade Runner 2049")


content='Emojis 🤖🏙️🔫 represent futuristic robots, urban setting, and action.'


In [4]:
invoke_chain("What is the first movie title?")


content='The first movie title is "The Grand Budapest Hotel."'
